In [16]:
import os
import tensorflow as tf
from IPython.display import YouTubeVideo
import pandas as pd
from PIL import Image





frame_dir = 'train/'
dataset = dict()
overall_dataset = []
max_features_vector = []
Y_train = dict()
X_train = dict()

print(os.listdir(frame_dir))
files=os.listdir(frame_dir)

i=0

for file in files:
         
    frame_lvl_record = frame_dir + file
    for example in tf.python_io.tf_record_iterator(frame_lvl_record):

        i=i+1
        
        tf_example = tf.train.Example.FromString(example)
        tf_seq_example = tf.train.SequenceExample.FromString(example)

        n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
        
        max_features_vector.append(n_frames)

        vid_id = tf_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8')
        vid_labels = tf_example.features.feature['labels'].int64_list.value    

        rgb_values = []
        audio_values = []
        for i in range(n_frames):
            rgb_values.append(tf_seq_example.feature_lists.feature_list['rgb']
                            .feature[i].bytes_list.value[0])
            audio_values.append(tf_seq_example.feature_lists.feature_list['audio']
                            .feature[i].bytes_list.value[0])
        sess = tf.InteractiveSession()
        rgb_frame = tf.decode_raw(rgb_values, tf.uint8).eval()
        audio_frame = tf.decode_raw(audio_values, tf.uint8).eval()
        sess.close()
        tf.reset_default_graph()

        dataset['id'] = vid_id
        dataset['labels'] = vid_labels
        dataset['rgb_frame'] = list(rgb_frame)
        dataset['audio_frame'] = list(audio_frame)
        overall_dataset.append(dataset)
        
datafr=pd.DataFrame(overall_dataset)

for data in overall_dataset:
    Y_train[data['id']]=data['labels']
    X_train["Video_ID"]=data['id']
    X_train["rgb_Features"]=data['rgb_frame']
    X_train["audio_Features"]=data['audio_frame']
        

        

print(len(overall_dataset))
train_y=pd.DataFrame(Y_train)
print(train_y)
print(len(X_train))
train_x=pd.DataFrame(X_train)
print(train_x)



['trainIc.tfrecord', 'trainpj.tfrecord']
1976
   eepj
0     3
1     4
2     6
3
    Video_ID                                       rgb_Features  \
0       eepj  [0, 72, 173, 30, 0, 0, 41, 33, 199, 132, 199, ...   
1       eepj  [0, 161, 155, 108, 0, 80, 106, 122, 145, 193, ...   
2       eepj  [29, 168, 231, 90, 0, 95, 64, 138, 106, 184, 1...   
3       eepj  [71, 162, 228, 133, 0, 127, 95, 156, 127, 159,...   
4       eepj  [24, 160, 255, 139, 0, 153, 89, 121, 109, 140,...   
5       eepj  [42, 159, 255, 121, 31, 174, 80, 124, 93, 122,...   
6       eepj  [39, 182, 254, 104, 89, 213, 94, 99, 21, 112, ...   
7       eepj  [23, 189, 255, 102, 128, 208, 77, 113, 19, 82,...   
8       eepj  [35, 186, 244, 92, 97, 201, 94, 101, 6, 106, 1...   
9       eepj  [42, 184, 236, 98, 110, 187, 91, 128, 40, 108,...   
10      eepj  [11, 192, 236, 117, 133, 198, 102, 109, 10, 92...   
11      eepj  [20, 191, 206, 111, 109, 188, 125, 103, 51, 13...   
12      eepj  [77, 173, 207, 96, 77, 198, 112, 77

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model = Sequential()


model.add(Conv2D(64, 3, 3, activation='relu', input_shape=(3800,15,2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


C:\Users\BUSINESS NETWORK\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", input_shape=(3800, 15,...)`
  import sys


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 3798, 13, 64)      1216      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3796, 11, 32)      18464     
_________________________________________________________________
flatten_5 (Flatten)          (None, 1336192)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                13361930  
Total params: 13,381,610
Trainable params: 13,381,610
Non-trainable params: 0
_________________________________________________________________
None
